### libraries and settings

In [1]:
import numpy as np
import pandas as pd 
from pylab import mpl, plt
import random
import optuna
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.set_num_threads(1)


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import warnings
warnings.simplefilter("ignore", UserWarning)

import vectorbtpro as vbt


from plotly.subplots import make_subplots
import plotly.graph_objects as go
from collections import deque


In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.use_deterministic_algorithms(True)

In [3]:
binary = True

In [4]:
# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device("cpu")

### data processing

In [5]:
df = pd.read_csv('2ySOLdata1hHss5.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)
pd.set_option('future.no_silent_downcasting', True)
df_filtered = df[df['signal'] != 'SignalNone']

# Iterate through the DataFrame and adjust the signals
for i in range(1, len(df_filtered)):
    current_signal = df_filtered.iloc[i]['signal']
    previous_signal = df_filtered.iloc[i - 1]['signal']
    current_close = df_filtered.iloc[i]['Close']
    previous_close = df_filtered.iloc[i - 1]['Close']
    
    if current_signal == previous_signal:
        if current_signal == 'SignalLong' and previous_close > current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        elif current_signal != 'SignalLong' and previous_close < current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        else:
            df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'


df.update(df_filtered)

if binary:
    # Assuming df is your DataFrame
    previous_signal = None  # Initialize a variable to keep track of the previous non-"SignalNone" value

    for i in range(len(df)):
        if df.iloc[i, df_filtered.columns.get_loc('signal')] == "SignalNone" and previous_signal is not None:
            df.iloc[i, df_filtered.columns.get_loc('signal')] = previous_signal  # Replace "SignalNone" with the previous signal
        elif df.iloc[i, df_filtered.columns.get_loc('signal')] != "SignalNone":
            previous_signal = df.iloc[i, df_filtered.columns.get_loc('signal')]  # Update the previous signal to the current one if it's not "SignalNone"

    df = df.loc[df['signal'] != 'SignalNone']

df['signal'] = df['signal'].replace({'SignalLong': 1, 'SignalShort': 0, 'SignalNone': 2})
df = df.ffill()

In [6]:
# vbt.IF.list_indicators("talib")

In [7]:
# vbt.phelp(vbt.talib("STOCHRSI").run)

### technical indicators

In [8]:
data = vbt.Data.from_data(df)

features = data.run("talib", mavp=vbt.run_arg_dict(periods=14))

# features = data.run(  
#     ["talib:RSI",
#      "talib:BBANDS",
#      "talib:ATR",
#      "talib:STOCHRSI",
#     ], 
#     talib_rsi=vbt.run_arg_dict(timeperiod=10),
#     talib_bbands=vbt.run_arg_dict(timeperiod=10),
#     talib_atr=vbt.run_arg_dict(timeperiod=14),
#     talib_stochrsi=vbt.run_arg_dict(timeperiod=10, fastk_period=6, fastd_period=9),
# )

data.data['symbol'] = pd.concat([data.data['symbol'], features], axis=1)
data.data['symbol'].drop(['Open', 'High', 'Low'], axis=1, inplace=True)
# This will drop columns from the DataFrame where all values are NaN
data.data['symbol'] = data.data['symbol'].dropna(axis=1, how='all')


data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = data.data['symbol'].drop('signal', axis=1).columns.tolist()


X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

X.columns = X.columns.astype(str)


### train test split

In [9]:
# Split the data into a training set and a test set
# Assuming X is a DataFrame or a NumPy array
indices = np.arange(X.shape[0])

# First, split your data into a training+validation set and a separate test set
X_train_val, X_test, y_train_val, y_test, indices_train_val, indices_test = train_test_split(X, y, indices, test_size=0.3, shuffle=False)

# Then, split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val, indices_train, indices_val = train_test_split(X_train_val, y_train_val, indices_train_val, test_size=0.2, shuffle=False)  # 0.2 here means 20% of the original data, or 25% of the training+validation set

# Now, `indices_val` holds the indices of your original dataset that were used for the validation set.


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.fit_transform(X_val)

In [10]:
timestep = 20

def create_sequences(input_data, timestep):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - timestep):
        seq = input_data[i:(i + timestep)]
        sequences.append(seq)
    return np.array(sequences)

X_train_list = create_sequences(X_train_scaled, timestep)
X_val_list = create_sequences(X_val_scaled, timestep)
X_test_list = create_sequences(X_test_scaled, timestep)
y_train_seq_ar = y_train[timestep:]
y_val_seq_ar = y_val[timestep:]
y_test_seq_ar = y_test[timestep:]

In [11]:
# Convert to numpy arrays
x_train_ar = np.array(X_train_list)
y_train_seq = np.array(y_train_seq_ar).astype(int)
x_val_ar = np.array(X_val_list)  
y_val_seq = np.array(y_val_seq_ar).astype(int)
x_test_ar = np.array(X_test_list)  
y_test_seq = np.array(y_test_seq_ar).astype(int)

In [12]:
# Convert to tensors
X_train_tensor = torch.tensor(x_train_ar, dtype=torch.float32)
X_val_tensor = torch.tensor(x_val_ar, dtype=torch.float32)
X_test_tensor = torch.tensor(x_test_ar, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train_seq, dtype=torch.long)
y_val_tensor = torch.tensor(y_val_seq, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_seq, dtype=torch.long)

if binary:
    y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val_seq, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32)

### class weights

In [13]:
if binary:
    class_weights_tensor = torch.tensor([1.0, 1.0, 1.0])
else:
    if isinstance(y_train_seq, torch.Tensor):
        y_train_seq_np = y_train_seq.cpu().numpy()
    else:
        y_train_seq_np = y_train_seq  # Assuming y_train_seq is already a numpy array or similar

    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_seq_np), y=y_train_seq_np)
    # Decrease the weight of the '2' class
    decrease_factor = 0.5  # Adjust this factor as needed
    class_weights[2] *= decrease_factor

    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)


    # Move class weights to the same device as your model and data
    class_weights_tensor = class_weights_tensor.to(device)  # device could be 'cpu' or 'cuda'


### model

In [14]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_output):
        # lstm_output shape: [batch_size, seq_length, hidden_dim]
        weights = torch.tanh(self.linear(lstm_output))
        weights = F.softmax(weights, dim=1)
        
        # Context vector with weighted sum
        context = weights * lstm_output
        context = torch.sum(context, dim=1)
        return context, weights

class BiLSTMClassifierWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_rate):
        super(BiLSTMClassifierWithAttention, self).__init__()
        
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        
        # Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim, kernel_size=3, stride=1, padding=1)
        
        # Batch Normalization Layer for Conv1d
        self.bn_conv1 = nn.BatchNorm1d(hidden_dim)
        
        # LSTM Layer
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # Attention Layer
        self.attention = Attention(hidden_dim * 2)  # For bidirectional LSTM
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)  # Adjusted for attention context vector
        
        # Batch Normalization Layer for FC1
        self.bn_fc1 = nn.BatchNorm1d(hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # Output layer
        
        # Additional Dropout for the fully connected layer
        self.dropout_fc = nn.Dropout(dropout_rate / 2)

    def forward(self, x):
        # Reshape x for Conv1d
        x = x.permute(0, 2, 1)
        
        # Convolutional layer
        x = self.conv1(x)
        x = self.bn_conv1(x)
        x = F.relu(x)
        
        # Reshape back for LSTM
        x = x.permute(0, 2, 1)
        
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        
        # LSTM layer
        out, _ = self.lstm(x, (h0, c0))
        
        # Applying attention mechanism to LSTM outputs
        context, _ = self.attention(out)
        
        # Fully connected layers using the context vector from attention
        out = self.fc1(context)
        out = self.bn_fc1(out)
        out = F.relu(out)
        out = self.dropout_fc(out)
        out = self.fc2(out)
        
        return out



### functions

In [15]:
def backtest(model, X_val_selected_gpu):
    # predictions and backtest
    with torch.no_grad():
        y_test_pred = model(X_val_selected_gpu)
        if binary:
            probabilities = torch.sigmoid(y_test_pred).squeeze()
            predicted_labels = (probabilities > 0.5).long()
        else:    
            probabilities = torch.softmax(y_test_pred, dim=1)
            _, predicted_labels = torch.max(probabilities, 1)
        predicted_labels_numpy = predicted_labels.cpu().numpy()

    # Use predicted labels to simulate a trading strategy
    adjusted_indices_val = indices_val[timestep:]
    adjusted_indices_test = indices_test[timestep:] 

    df_split = data.data['symbol'].iloc[adjusted_indices_val].copy()
    df_split.loc[:, "signal"] = predicted_labels_numpy
    signal = df_split['signal']
    entries = signal == 1
    exits = signal == 0
    pf = vbt.Portfolio.from_signals(
        close=df_split.Close, 
        long_entries=entries, 
        short_entries=exits,
        size=100,
        size_type='value',
        init_cash='auto'
    )
    stats = pf.stats()
    total_return = round(stats['Total Return [%]'], 2)
    vbt.settings.set_theme('dark')
    vbt.settings['plotting']['layout']['width'] = 600
    vbt.settings['plotting']['layout']['height'] = 200
    pf.plot({"orders", "cum_returns"}).show()
    print(f"Total Return: {total_return}%")

In [16]:
def plot_target():
    adjusted_indices_val = indices_val[timestep:]
    adjusted_indices_test = indices_test[timestep:] 

    df_split = data.data['symbol'].iloc[adjusted_indices_val].copy()
    df_split.loc[:, "signal"] = y_val_tensor
    signal = df_split['signal']
    entries = signal == 1
    exits = signal == 0
    pf = vbt.Portfolio.from_signals(
        close=df_split.Close, 
        long_entries=entries, 
        short_entries=exits,
        size=100,
        size_type='value',
        init_cash='auto'
    )
    stats = pf.stats()
    vbt.settings.set_theme('dark')
    vbt.settings['plotting']['layout']['width'] = 600
    vbt.settings['plotting']['layout']['height'] = 200
    pf.plot({"orders", "cum_returns"}).show()

In [17]:
def validate_multi_with_metrics(model, criterion, X_val, y_val, device):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model(X_val)
        predicted_probs = torch.softmax(output, dim=1)
        predictions = torch.argmax(predicted_probs, dim=1)  # Get the index of the max log-probability as the prediction
        
        loss = criterion(output, y_val)  # Ensure y_val is of dtype long and contains class indices

        # Convert to CPU and numpy for sklearn metrics
        predictions_np = predictions.cpu().numpy()
        y_val_np = y_val.cpu().numpy()

        accuracy = accuracy_score(y_val_np, predictions_np)
        precision = precision_score(y_val_np, predictions_np, average='weighted')
        recall = recall_score(y_val_np, predictions_np, average='weighted')
        f1 = f1_score(y_val_np, predictions_np, average='weighted')

    model.train()  # Set back to train mode
    return {
        'loss': round(loss.item(), 4),
        'accuracy': round(accuracy, 4),
        'precision': round(precision, 4),
        'recall': round(recall, 4),
        'f1': round(f1, 4)
    }

In [18]:


def validate_binary_with_metrics(model, criterion, X_val, y_val, device):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model(X_val)
        predicted_probs = torch.sigmoid(output)
        predictions = (predicted_probs > 0.5).float()  # Apply threshold to get binary predictions
        loss = criterion(output, y_val.view_as(output))

        # Convert to CPU and numpy for sklearn metrics
        predictions_np = predictions.cpu().numpy()
        y_val_np = y_val.cpu().numpy()
        predicted_probs_np = predicted_probs.cpu().numpy()

        accuracy = (predictions.view_as(y_val) == y_val).sum().item() / len(y_val)
        precision = precision_score(y_val_np, predictions_np)
        recall = recall_score(y_val_np, predictions_np)
        f1 = f1_score(y_val_np, predictions_np)
        auc = roc_auc_score(y_val_np, predicted_probs_np)  # Use probabilities for AUC

    model.train()  # Set back to train mode
    return {
        'loss': round(loss.item(),4),
        'accuracy': round(accuracy,4),
        'precision': round(precision,4),
        'recall': round(recall,4),
        'f1': round(f1,4),
        'auc': round(auc,4)

    }

In [19]:
def validate_financials(model, X_val_selected_gpu):
    model.eval()
    with torch.no_grad():
        y_test_pred = model(X_val_selected_gpu)
        if binary:
            probabilities = torch.sigmoid(y_test_pred).squeeze()
            predicted_labels = (probabilities > 0.5).long()
        else:    
            probabilities = torch.softmax(y_test_pred, dim=1)
            _, predicted_labels = torch.max(probabilities, 1)
        predicted_labels_numpy = predicted_labels.cpu().numpy()

    # Use predicted labels to simulate a trading strategy
    adjusted_indices_val = indices_val[timestep:]
    adjusted_indices_test = indices_test[timestep:] 
    
    df_split = data.data['symbol'].iloc[adjusted_indices_val].copy()
    df_split.loc[:, "signal"] = predicted_labels_numpy
    signal = df_split['signal']
    entries = signal == 1
    exits = signal == 0
    pf = vbt.Portfolio.from_signals(
        close=df_split.Close, 
        long_entries=entries, 
        short_entries=exits,
        size=100,
        size_type='value',
        init_cash='auto'
    )
    stats = pf.stats()
    total_return = stats['Total Return [%]']
    orders = stats['Total Orders']
    calmer_ratio = stats['Calmar Ratio']
    
    model.train()
    return {
        "orders": orders,
        "calmer_returns": (calmer_ratio+total_return)
    }

In [20]:
def plot_vals(epoch_nums_1, val_loss, accuracy, precision, recall, f1):
    # Create a subplot with 2 rows and 1 column
    fig = make_subplots(rows=2, cols=1)

    # Add validation loss trace to the first row
    fig.add_trace(go.Scatter(x=epoch_nums_1, y=val_loss, mode='lines+markers', name='val_loss'), row=1, col=1)
    
    # Add validation metrics traces to the second row
    fig.add_trace(go.Scatter(x=epoch_nums_1, y=accuracy, mode='lines+markers', name='accuracy'), row=2, col=1)
    fig.add_trace(go.Scatter(x=epoch_nums_1, y=precision, mode='lines+markers', name='precision'), row=2, col=1)
    fig.add_trace(go.Scatter(x=epoch_nums_1, y=recall, mode='lines+markers', name='recall'), row=2, col=1)
    fig.add_trace(go.Scatter(x=epoch_nums_1, y=f1, mode='lines+markers', name='f1'), row=2, col=1)

    # Update layout for the combined figure
    fig.update_layout(
        template='plotly_dark',
        autosize=False,
        width=700,  # Adjust the width of the figure
        height=300,  # Adjust the height of the figure (make it larger to accommodate both subplots)
        title_text='loss & metrics over epochs',
        title_font_size=10,
        margin=dict(l=5, r=5, b=5, t=30, pad=5),
        legend=dict(
            font=dict(
                size=5)
        )
    )

    # Show the combined figure
    fig.show()

In [21]:
def plot_returns(epoch_nums_1, calmer_return):
    return_metrics_fig = go.Figure()

    return_metrics_fig.add_trace(go.Scatter(x=epoch_nums_1, y=calmer_return, mode='lines+markers', name='calmer returns'))

    return_metrics_fig.update_layout(
        template='plotly_dark',
        autosize=False,
        width=700,  # Set the width of the figure
        height=150,  # Set the height of the figure
        title='calmer ratio + returns over epochs',  # You can set the title directly here
        title_font_size=10,
        margin=dict(l=5, r=5, b=5, t=30, pad=5),
        legend=dict(
            font=dict(
                size=5)
        )
    )
    return_metrics_fig.show()

In [22]:
hidden_dim = 32
num_layers = 2

learning_rate=0.5
step_size=10
gamma=0.8
dropout_rate=0

variance_threshold = 3
print_epochs = 30
rolling_window_size = 15
num_epochs = 2000

num_trials = 2

# lets batch our indicators in groups of 20, and do 3 backtests of each

### study training loop

In [23]:
def objective(trial):
    
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    torch.use_deterministic_algorithms(True)


    # feature_idx = trial.suggest_int('feature_idx', 0, X_train_tensor.shape[2] - 1)
    X_train_selected = X_train_tensor[:, :, 20:39]
    X_val_selected = X_val_tensor[:, :, 20:39]
    # X_test_selected = X_test_tensor[:, :, feature_idx:feature_idx+1]
    
    # Move tensors to the MPS device
    X_train_selected_gpu = X_train_selected.float().to(device)
    X_val_selected_gpu = X_val_selected.float().to(device)
    y_train_tensor_gpu = y_train_tensor.long().to(device)
    y_val_tensor_gpu = y_val_tensor.long().to(device)
    
    out_dims = len(np.unique(y_train_tensor.cpu().numpy()))
    if binary:
        out_dims = 1
        y_train_tensor_gpu = y_train_tensor.float().to(device)
        y_val_tensor_gpu = y_val_tensor.float().to(device)
    
    # Create the model with bidirectional LSTM
    model = BiLSTMClassifierWithAttention(input_dim=X_train_selected.shape[-1], hidden_dim=hidden_dim, num_layers=num_layers, output_dim=out_dims, dropout_rate=dropout_rate).to(device)
    optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimiser, step_size=step_size, gamma=gamma)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    if binary:
        criterion = nn.BCEWithLogitsLoss()
        y_val_tensor_gpu = y_val_tensor_gpu.unsqueeze(1)
        


    rolling_window = deque(maxlen=rolling_window_size)
    epoch_nums_1 = []
    calmer_return = []
    val_loss = []
    accuracy = []
    precision = []
    recall = []
    f1 = []        
   
    model.train()
    for epoch in range(num_epochs):
        optimiser.zero_grad()   
        output = model(X_train_selected_gpu)
        output = torch.squeeze(output)  # This removes the extra dimension
        loss = criterion(output, y_train_tensor_gpu)  # Ensure y_train is of type torch.long
        loss.backward()
        optimiser.step()
        scheduler.step()


        if epoch % print_epochs == 0:  # Adjust as needed

            financial_results = validate_financials(model, X_val_selected_gpu)
            
            validation_results = validate_multi_with_metrics(model, criterion, X_val_selected_gpu, y_val_tensor_gpu, device)
            if binary:
                validation_results = validate_binary_with_metrics(model, criterion, X_val_selected_gpu, y_val_tensor_gpu, device)
            epoch_nums_1.append(epoch)
            accuracy.append(validation_results['accuracy'])
            val_loss.append(validation_results['loss'])
            precision.append(validation_results['precision'])
            recall.append(validation_results['recall'])
            f1.append(validation_results['f1'])
            calmer_return.append(financial_results['calmer_returns'])
            rolling_window.append(financial_results['calmer_returns'])
            
            if len(rolling_window) == rolling_window_size:
                variance = np.var(list(rolling_window))
                
                if variance < variance_threshold:
                    print(f"Early stopping triggered epoch {epoch}. Variance is below the threshold.")
                    break
                if len(calmer_return) >= rolling_window_size and all(x < 0 for x in calmer_return[-rolling_window_size:]):
                    print(f"Early stopping triggered epoch {epoch}. The last {rolling_window_size} returns are negative.")
                    break
            

    plot_vals(epoch_nums_1, val_loss, accuracy, precision, recall, f1)
    plot_returns(epoch_nums_1, calmer_return)
    backtest(model, X_val_selected_gpu)
    return loss.item()

sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=num_trials)

print('Best trial:', study.best_trial.params)


[I 2024-03-03 12:48:33,818] A new study created in memory with name: no-name-589dc2a4-b21d-4d76-a6a2-c8826b54e257


Early stopping triggered epoch 540. Variance is below the threshold.


[I 2024-03-03 12:50:21,622] Trial 0 finished with value: 0.6913487911224365 and parameters: {}. Best is trial 0 with value: 0.6913487911224365.


Total Return: 15.3%
Early stopping triggered epoch 510. Variance is below the threshold.


[I 2024-03-03 12:51:59,388] Trial 1 finished with value: 0.6908795833587646 and parameters: {}. Best is trial 1 with value: 0.6908795833587646.


Total Return: 21.76%
Best trial: {}


### end